In [4]:
import numpy as np
import pandas as pd # csv files
import random

iris=pd.read_csv('Iris.csv')

In [8]:
class NaiveBayes:
    #obliczanie średniej
    @staticmethod
    def mean(atr):
        return (sum(atr)/len(atr))
        
    #odchylenie standardowe
    @staticmethod
    def std(atr,mean): 
        sumVal = 0 
        for i in atr:
            sumVal+=((i-mean)**2)
        
        return np.sqrt(sumVal/len(atr))
        
    #funkcja gaussa
    @staticmethod
    def gauss(x,mean,std):
        # x - jedna z wartości sample [a,b,c,d]
        exponent = np.exp(-(x-mean)**2/(2*std**2))
            
        return (1/(np.sqrt(2*np.pi*std**2))*exponent)
        
    def classify(x, sample, types):
        
        #separacja klas z x
        #schamet data: data[variety][property][value]
        values = iris.select_dtypes(exclude="object")
        indexes = values.columns.tolist()

        data = []
        means = []
        stds = []
        for i in range(len(types)):
            data.append([])
            means.append([])
            stds.append([])
           
        for i in types:
            ind = types.index(i)
            tmp = np.where(x.variety == i)
            for v in indexes:
                dex = indexes.index(v)
                data[ind].append([])
                means[ind].append([])
                stds[ind].append([])
                for j in tmp[0]:
                    data[ind][dex].append(x.iloc[j][v])
                #obliczenie średniej i odchylenia dla każdego atrybutu dla każdej klasy
                means[ind][dex]=NaiveBayes.mean(data[ind][dex])
                stds[ind][dex]=NaiveBayes.std(data[ind][dex],means[ind][dex])
        #obliczenie prawdopodobieństw wystąpienia każdego atrybutu dla każdej klasy - gauss
        finales = [1]*len(types)
        for i in range(len(types)):
            for j in range(len(indexes)):
                finales[i]*= NaiveBayes.gauss(sample[j],means[i][j],stds[i][j])
        
        #zwrócenie klasy z max prawdopodobieństwem
        return types[finales.index(max(finales))]

In [6]:
class ProcessingData:
    @staticmethod
    def stiffle(x):
        for i in range(len(x)-1,0,-1):
            tmp = random.randint(0,i)
            x.iloc[i], x.iloc[tmp] = x.iloc[tmp], x.iloc[i]
        return x
    
    @staticmethod
    def normalize(X):
        x=X.copy()
        values = x.select_dtypes(exclude="object")
        indexes = values.columns.tolist()
        
        for i in indexes:
            xmax = max(x[i])
            xmin = min(x[i])
            mnx = xmax - xmin
            for v in range(len(x[i])):
                x.at[v,i] = (x.at[v,i]-xmin)/mnx
        return x    
        
    @staticmethod
    def split(x,a):
        n = len(x)
        tabA=x[0:int(a*n)]
        tabB=x[int(a*n):n]
        return tabA, tabB

In [7]:
IrisMixed=ProcessingData.normalize(ProcessingData.stiffle(iris))
irisTrain, irisVal = ProcessingData.split(IrisMixed,0.7)
types = ['Setosa','Versicolor','Virginica']
accuracy=0;
for i in irisVal.iloc:
    sample = [i['sepal.length'],i['sepal.width'],i['petal.length'],i['petal.width']] 
    answear = NaiveBayes.classify(irisTrain,sample,types)
    #print(answear,'-',i.variety)
    if(answear == i.variety):
        accuracy+=1
print('accuracy = ',(accuracy/len(irisVal)*100),'%')

accuracy =  93.33333333333333 %
